In [2]:
import pandas as pd
import sys
sys.path.append("..")

from src.llm_eval.utils import process_responses, collect_fit_rank_data, compute_correlations_one

In [77]:
llm_data_paths = {
    "wiki": {
        "gpt4o": "../data/llm_out/wiki/q1_then_q3_dspy,q1_then_q2_dspy_gpt-4o-mini-2024-07-18_20241213_135628",
        "llama3.1": "../data/llm_out/wiki/q1_then_q3_dspy,q1_then_q2_dspy_llama3.1:8b-instruct-q8_0_20241213_135530"
    },
    "bills": {
        "gpt4o": "../data/llm_out/bills/q1_then_q3_dspy,q1_then_q2_dspy_gpt-4o-mini-2024-07-18_20241213_141345",
        "llama3.1": "../data/llm_out/bills/q1_then_q3_dspy,q1_then_q2_dspy_llama3.1:8b-instruct-q8_0_20241213_133218"
    }
}

data_jsons = [
    "../data/json_out/config_pilot_wiki.json",
    "../data/json_out/config_pilot_wiki_part2.json",
    #"../data/json_out/config_bills_part1.json"
]

response_csvs = [
   # "../data/human_annotations/Cluster+Evaluation+-+Sort+and+Rank+-+Bills_December+14,+2024_03.35.csv",
    "../data/human_annotations/Cluster+Evaluation+-+Sort+and+Rank_December+12,+2024_05.19.csv",
]

start_date = "2024-12-06 09:00:00"

In [78]:
responses = {}
for csv in response_csvs:
    for topic_id, topic_responses in process_responses(csv, data_jsons, start_date=start_date, path_save=None, removal_condition="loose").items():
        if topic_responses:
            responses[topic_id] = topic_responses

Total responses: 142
Total responses: 142
Removed: 25


In [81]:
dataset = "wiki"
model = "gpt4o"
topic_model = "mallet"
path = llm_data_paths[dataset][model]

q1_data = pd.read_json(f"{path}/llm_results_q2.json").to_dict(orient="records")
q1_data_df = pd.DataFrame(q1_data)
q1_data_df["labels"] = q1_data_df["labels"].apply(lambda x: x[0])
q1_data_df["user_cats"] = q1_data_df["id"].apply(
    lambda x: [r["category"] for r in responses[x]] if x in responses else []
)

q1_data_df["top_wrds"] = q1_data_df["id"].apply(lambda x: " ".join(responses[x][0]["topic_words"][:5]))
q1_data_df = q1_data_df[(q1_data_df.id == "wikitext-labeled/mallet/44") | (q1_data_df.id == "wikitext-labeled/ctm/2") | (q1_data_df.id == "wikitext-labeled/bertopic/7")]
#q1_data_df = q1_data_df[(q1_data_df.id == "bills-labeled/mallet/4") | (q1_data_df.id == "bills-labeled/ctm/30") | (q1_data_df.id == "bills-labeled/bertopic/40")]
q1_data_df[["id", "model", "labels", "user_cats", "top_wrds"]]

,id,model,labels,user_cats,top_wrds
6,wikitext-labeled/bertopic/7,bertopic,Sports and Athletics,"[Professional Basketball and Baseball Players,...",yard season team yards league
11,wikitext-labeled/ctm/2,ctm,Major League Baseball Players and Achievements,"[Former MLB players, American baseball league,...",career hit games season league
21,wikitext-labeled/mallet/44,mallet,Major League Baseball Players and History,"[Professional baseball players, American profe...",season game games home runs


In [84]:
q1_data_df[["id", "model", "labels", "user_cats", "top_wrds"]].iloc[0].user_cats

['Professional Basketball and Baseball Players',
 'American sports and their associated famous sportsmen',
 'Sports champions',
 'American sports history']

In [ ]:
# BILLS
#Sampled Matches (Original Topic IDs): 
# [[(0, 4), (1, 30), (2, 40)], 
# [(0, 40), (1, 3), (2, 1)], [(0, 23), (1, 18), (2, 18)], [(0, 12), (1, 27), (2, 31)], [(0, 19), (1, 34), (2, 3)], [(0, 44), (1, 23), (2, 27)], [(0, 1), (1, 28), (2, 13)], [(0, 17), (1, 11), (2, 19)]]

# WIKI
#Sampled Matches (Original Topic IDs):
# [[(0, 16), (1, 35), (2, 47)],
# [(0, 44), (1, 2), (2, 7)], 
# [(0, 49), (1, 1), (2, 8)], [(0, 38), (1, 3), (2, 36)]]

In [3]:
all_df_cohr = pd.concat([pd.read_csv("../data/all_cohrs_bills.csv"), pd.read_csv("../data/all_cohrs_wiki.csv")])

for dtset in ["bills", "wiki"]:
    print(f"\n\nDataset: {dtset}")
    
    df_cohr = all_df_cohr[all_df_cohr["id"].str.contains(dtset)]

    df_cohr_mallet = df_cohr[df_cohr["id"].str.contains("mallet")]
    df_cohr_ctm = df_cohr[df_cohr["id"].str.contains("ctm")]
    df_cohr_bertopic = df_cohr[df_cohr["id"].str.contains("bertopic")]

    mean_mallet = df_cohr_mallet.drop(columns=["id"]).mean()
    mean_ctm = df_cohr_ctm.drop(columns=["id"]).mean()
    mean_bertopic = df_cohr_bertopic.drop(columns=["id"]).mean()

    # Imprimir las medias
    print("Mallet:")
    print(mean_mallet)
    print("\n CTM:")
    print(mean_ctm)
    print("\nBERTopic:")
    print(mean_bertopic)
    
    print("#"*50)




Dataset: bills
Mallet:
npmi    0.070198
cv      0.520695
dtype: float64

 CTM:
npmi    0.106991
cv      0.670567
dtype: float64

BERTopic:
npmi    0.071326
cv      0.554651
dtype: float64
##################################################


Dataset: wiki
Mallet:
npmi    0.125730
cv      0.686326
dtype: float64

 CTM:
npmi    0.134351
cv      0.709896
dtype: float64

BERTopic:
npmi    0.150289
cv      0.751968
dtype: float64
##################################################
